In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, 
                                     AveragePooling2D, MaxPooling2D)
from tensorflow.keras.initializers import glorot_uniform, random_uniform
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools

# Configuration
dataset_url = r'C:\Users\abudh\Desktop\CropWatch\EuroSAT\2750'
model_path = r'C:\Users\abudh\Desktop\CropWatch\Training_model.h5'

batch_size = 32
img_height = 64
img_width = 64
validation_split = 0.2
rescale = 1.0 / 255
epochs = 1  # Adjust the number of epochs as needed

# Identity block for ResNet
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    return X

# Convolutional block for ResNet
def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

# ResNet model definition
def ResNet50(input_shape=(64, 64, 3), classes=10):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    X = AveragePooling2D(pool_size=(2, 2))(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=glorot_uniform(seed=0))(X)
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model

# Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('accuracy')
    ax1.set_title('Accuracy over epochs')
    ax1.legend(['Train', 'Validation'], loc='upper left')
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('loss')
    ax2.set_title('Loss over epochs')
    ax2.legend(['Train', 'Validation'], loc='upper right')
    plt.show()

# Prepare dataset
def prepare_data(dataset_url, img_height, img_width, batch_size, validation_split, rescale):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=validation_split, rescale=rescale)
    train_dataset = datagen.flow_from_directory(directory=dataset_url, target_size=(img_height, img_width), 
                                                batch_size=batch_size, subset="training", class_mode='categorical')
    test_dataset = datagen.flow_from_directory(directory=dataset_url, target_size=(img_height, img_width), 
                                               batch_size=batch_size, subset="validation", class_mode='categorical')
    return train_dataset, test_dataset, list(train_dataset.class_indices.keys())

# Train model
def train_model(train_dataset, test_dataset, class_names, epochs=epochs):
    model = ResNet50(input_shape=(img_height, img_width, 3), classes=len(class_names))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)
    model.save(model_path)
    plot_training_history(history)
    return model, history

# Evaluate model
def evaluate_model(model, test_dataset, class_names):
    y_pred = []
    y_true = []
    
    # Iterate over the test dataset
    for images, labels in test_dataset:
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))
        
        if len(y_true) >= test_dataset.samples:
            break

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_names)

# Plot confusion matrix
def plot_confusion_matrix(cm, class_names, figsize=(10, 7)):
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    fmt = 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="black")
    plt.show()



In [7]:

# Main execution
train_dataset, test_dataset, class_names = prepare_data(dataset_url, img_height, img_width, batch_size, validation_split, rescale)


Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.


In [ ]:

model, history = train_model(train_dataset, test_dataset, class_names)
evaluate_model(model, test_dataset, class_names)

Epoch 1/2


C:\Users\abudh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


675/675 ━━━━━━━━━━━━━━━━━━━━ 917s 1s/step - accuracy: 0.4517 - loss: 1.9825 - val_accuracy: 0.4456 - val_loss: 1.7194
Epoch 2/2
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6339 - loss: 1.1282